<b>Imports. The "limit" parameter in keyedvectors.load_word2vec_format will give you the n most common words </b>

In [1]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format(r"C:\Users\mikef\Downloads\GoogleNews-vectors-negative300.bin.gz", binary=True, limit=15000)

<b>Run nltk.download() to download brown and wordnet. Only needs to be done once.</b>

In [2]:
import nltk
#nltk.download()

In [3]:
import random
import inflect
from nltk.corpus import brown
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")

Some commented out test code

In [5]:
#old code for getting sentences in serial that contain inputted word
# def sentence_word(word):
#     word_sents = []
#     for i in brown.sents(categories='fiction'):
#         if word in i:
#             word_sents.append(i)
#     return word_sents

In [6]:
#word_vectors.most_similar(positive=['said'])

In [7]:
#1 - word_vectors.distance('solicitation','envelope')

<h1>Main Functions</h1>

<p>Generates a list of "semantic" "opposites" of a word. List items are randomized so that not every instance of a word, say 'dog', has the same opposite on each occurence. </p>

In [58]:
def opposite_vector(word):
    try:
        zz = word_vectors.most_similar(negative=[word],topn=1000)
        random.shuffle(zz)
        return zz
    except:
        return 0

In [54]:
az =(word_vectors.most_similar(positive=['king'],topn=100))
random.shuffle(az)
print(az)

[('god', 0.4467619061470032), ('ultimate', 0.27755099534988403), ('billionaire', 0.2469705492258072), ('genius', 0.2825397849082947), ('nephew', 0.2755177319049835), ('colonial', 0.24549832940101624), ('Princess', 0.321186363697052), ('mighty', 0.3483976423740387), ('icon', 0.3241482079029083), ('winner', 0.24184319376945496), ('Prime_Minister', 0.29721054434776306), ('golden', 0.315428227186203), ('republic', 0.3394545912742615), ('coup', 0.272361695766449), ('epic', 0.24486184120178223), ('champ', 0.37749436497688293), ('monster', 0.34152844548225403), ('Pop', 0.2838295102119446), ('masters', 0.2596246600151062), ('bishop', 0.3010614216327667), ('kingdom', 0.5210406184196472), ('legends', 0.25968819856643677), ('him', 0.2486790269613266), ('shrine', 0.2691285312175751), ('King', 0.5158917903900146), ('lion', 0.24740149080753326), ('lover', 0.29147082567214966), ('famed', 0.26052629947662354), ('Prince', 0.362155944108963), ('minister', 0.26343467831611633), ('favorite', 0.24856851994

<p>Replaces words in a sentence with their "semantic" "opposites". Only replaces words of selected part of speech categories. Input the sentence as a list, and input the desired part of speech categories as a list as well. POS categories must be inputed as follows: Noun = 'NOUN', Verb = 'VERB', Adjective = 'ADJ'. Example: opposite_sentence(['Look', 'at', 'the', 'funny', 'cat'], ['NOUN','ADJ']) will try to switch all the nouns and adjectives in the sentence "Look at the funny cat" with their opposites. </p>

In [9]:
def opposite_sentence(sentence,cats=['NOUN']): #sentence inputted as list, cats = pos categories
    a = nlp(' '.join(sentence.copy())) #a = list-like object of words that can now be queried for their POS (eg a[0].pos_)
    return_sent = sentence.copy()
    replacements = []
    
    
    ###IF THERE'S A PART IN a THEN IT WILL HAVE A LARGER RANGE
    
    
    for i in range(len(a)): #for each word (or punctuation) in sentence
        pos = a[i].pos_     #get POS of word 
        if pos not in cats: #If POS not in the inputed categories then ignore it
            continue
        opposite_list = opposite_vector(str(a[i])) #Get the words list of opposite words
        if opposite_list == 0:                     #Unless we can't find a list of opposite words  
            continue
#Now iterate over each opposite word. Try replacing it with the original word in the sentence.
#If the opposite word takes on the same POS when replaced into the sentence, finalize it as the official replacement word
        for opposite in [o[0] for o in opposite_list]: 
            c = return_sent.copy() #get current return sentence
            c[i] = opposite        #replace current word with opposite
#This code may be useful if theres issues with singular/plural nouns -- will look into more later
#             if inflect.engine().singular_noun(opposite) != False:
#                 continue     
            if nlp(' '.join(c))[i].pos_== pos: #check if replaced opposite has same POS or original word
                replacements.append((i,opposite)) #if so, officially replace it
                break
    for i in replacements: 
        return_sent[i[0]] = "[" + i[1] + "]"
    return return_sent

In [44]:
def opposite_sentence(sentence,cats=['NOUN']): #sentence inputted as list, cats = pos categories
    a = nlp(' '.join(sentence.copy())) #a = list-like object of words that can now be queried for their POS (eg a[0].pos_)
    return_sent = sentence.copy()
    replacements = []
    print(len(a))
    print(len(sentence))
    
    ###IF THERE'S A PART IN a THEN IT WILL HAVE A LARGER RANGE
    skip = 0
    
    for i in range(len(a)): #for each word (or punctuation) in sentence
        pos = a[i].pos_     #get POS of word 
        if pos == 'PART':
            skip +=1
        if pos not in cats: #If POS not in the inputed categories then ignore it
            continue
        opposite_list = opposite_vector(str(a[i])) #Get the words list of opposite words
        if opposite_list == 0:                     #Unless we can't find a list of opposite words  
            continue
        print(i)
#Now iterate over each opposite word. Try replacing it with the original word in the sentence.
#If the opposite word takes on the same POS when replaced into the sentence, finalize it as the official replacement word
        for opposite in [o[0] for o in opposite_list]: 
            c = return_sent.copy() #get current return sentence 
            c[i-skip] = opposite        #replace current word with opposite
#This code may be useful if theres issues with singular/plural nouns -- will look into more later
#             if inflect.engine().singular_noun(opposite) != False:
#                 continue     
            if nlp(' '.join(c))[i].pos_== pos: #check if replaced opposite has same POS or original word
                replacements.append((i-skip,opposite)) #if so, officially replace it
                break
    for i in replacements: 
        print(return_sent)
        print(i[0])
        print(replacements)
        return_sent[i[0]] = "[" + i[1] + "]" #i[0] is index, i[1] is string replacement
    return return_sent

In [11]:
for i in nlp('hi there, everybody\'s dancing'):
    print(i,i.pos_)

hi INTJ
there ADV
, PUNCT
everybody PRON
's PART
dancing NOUN


In [35]:
nlp(qq).text

"hi there , everybody's dancing"

In [32]:
qq = 'hi there , everybody\'s dancing'

In [49]:
for i in qq:
    print(i,i._pos)

AttributeError: 'str' object has no attribute '_pos'

In [61]:
qq ='hi there dog , everybody\'s dancing'.split()
print(qq[5])
opposite_sentence(qq)

dancing
7
6
2
6
['hi', 'there', 'dog', ',', "everybody's", 'dancing']
2
[(2, 'reservations'), (5, 'efficiency')]
['hi', 'there', '[reservations]', ',', "everybody's", 'dancing']
5
[(2, 'reservations'), (5, 'efficiency')]


['hi', 'there', '[reservations]', ',', "everybody's", '[efficiency]']

<h1>Display Function(s)</h1>

In [ ]:
def print_x(total_sents,cats):
    for i in range(total_sents):
        q = random.choice(brown.sents())
        try:
            a = ' '.join(q)
            b = ' '.join(opposite_sentence(q,cats))
            if a != b:
                print('*********************************************')
                print(a)
                print()
                print(b)
                print('*********************************************')
        except:
            pass